In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train/train.csv")
test = pd.read_csv("../input/test/test.csv")
breed_labels = pd.read_csv("../input/breed_labels.csv")
color_labels = pd.read_csv("../input/color_labels.csv")
state_labels = pd.read_csv("../input/state_labels.csv")
submission = pd.read_csv("../input/test/sample_submission.csv")

In [ ]:
train.head()



In [ ]:
breed_labels.head()


In [ ]:
color_labels.head()


In [ ]:
state_labels.head()


As we'll be doing EDA on training set. We need to change the numerical notation with text.


In [ ]:
#Copy training set to train_eda
train_ed = train.copy()
train_ed.head()

In [ ]:
# Joining train_eda with state labels, breed_labels and color labels datasets
breed_labels.rename(columns={'BreedID':'Breed1'}, inplace=True)
state_labels.rename(columns={'StateID':'State'}, inplace=True)
train_ed = train_ed.merge(state_labels,on="State",how='left')
train_ed = train_ed.merge(breed_labels,on="Breed1",how='left')
train_ed.shape

In [ ]:
train_ed.rename(columns={'BreedName':'MainBreed'}, inplace=True)
train_ed.rename(columns={'Type_x':'Type'}, inplace=True)

In [ ]:
train_ed= train_ed.drop(["Breed1",'Type_y'],axis = 1)
train_ed.columns

In [ ]:
breed_labels.rename(columns={'Breed1':'Breed2',"BreedName":"SecondaryName"},inplace = True)


In [ ]:
train_ed=train_ed.merge(breed_labels,on="Breed2",how="left")
train_ed.rename(columns={'Type_x':'Type'},inplace = True)
train_ed=train_ed.drop(["Type_y",'Breed2'],axis = 1)
train_ed.columns
train_ed.shape

In [ ]:
color_labels.rename(columns={'ColorID':'Color1','ColorName':'ColorName1'},inplace = True)
train_ed=train_ed.merge(color_labels,on='Color1',how='left')
train_ed.shape

In [ ]:
color_labels.rename(columns={'Color1':'Color2','ColorName1':'ColorName2'},inplace = True)


In [ ]:
train_ed=train_ed.merge(color_labels,on='Color2',how='left')
train_ed.columns

In [ ]:
color_labels.rename(columns={'Color2':'Color3','ColorName2':'ColorName3'},inplace = True)
train_ed=train_ed.merge(color_labels,on='Color3',how='left')

In [ ]:

train_ed = train_ed.drop(["Color1",'Color2','Color3'],axis = 1)
train_ed.columns


In [ ]:
train_ed.shape

In [ ]:
train_ed = train_ed.drop(["State"],axis = 1)


In [ ]:
train_ed.head()

In [ ]:
train_ed["Gender"]=train_ed["Gender"].replace({1:'Male',2:'Female',3:'Mixed'})
train_ed["MaturitySize"]=train_ed["MaturitySize"].replace({1:'Small',2:'Medium',3:'Large',4:'Extra Large',0:"Not Specified"})
train_ed["FurLength"]=train_ed["FurLength"].replace({1:'Short',2:'Medium',3:'Long',0:"Not Specified"})
train_ed["Vaccinated"]=train_ed["Vaccinated"].replace({1:'Yes',2:'No',3:'Not Sure'})
train_ed["Dewormed"]=train_ed["Dewormed"].replace({1:'Yes',2:'No',3:'Not Sure'})
train_ed["Sterilized"]=train_ed["Sterilized"].replace({1:'Yes',2:'No',3:'Not Sure'})
train_ed["Health"]=train_ed["Health"].replace({1:'Healthy',2:'Minor Injury',3:'Major Injury',0:"Not Specified"})
train_ed["Type"]=train_ed["Type"].replace({1:'Dog',2:'Cat'})
train_ed["AdoptionSpeed"]=train_ed["AdoptionSpeed"].replace({0:'Adopted on the same day',1:'Adopted within 1 week',2:'Adopted within 1 month',3:'Adopted within 2-3 months',4:'Didnt get adopted'})
train_ed.head()


Now that we have converted our data for EDA purpose, let's drive insights from the same.
First we'll see whether which animal has more listings in the set.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g=sns.countplot(x="Type",data=train_ed,palette=sns.color_palette("muted"))
#Turns out cats are listed more on the site
plot_dict={}
val_counts=dict(train_ed["Type"].value_counts().sort_index())
for k,v in val_counts.items():
    if k not in plot_dict:
        plot_dict[val_counts[k]] = val_counts[k]
    else:
        plot_dict[0] = 0
for x in g.patches:
    height = x.get_height()
    g.text(x.get_x() + x.get_width()/2.0,height,plot_dict[height],ha = "center", va = "bottom", fontsize = 8,weight="semibold",size="large")



We can see that more number of dogs are listed on the website.
Now let's see what percentage of Cat/Dog are getting adopted, and at what speed.

In [ ]:
#function to create AdoptionSpeed plots using a grouper 
import math
def make_plot (df,grouper,col='AdoptionSpeed',title='',palette = ''):
    g=sns.countplot(x=col,hue=grouper,data=df,palette=palette) 
    g.set_xticklabels(labels = df[col].unique(),rotation=-60,ha='center')
    g.set_ylabel("Number of Adoptions")
    g.set_title(title)
    g.legend(frameon=False)
    sns.set_style({"axes.facecolor":"white"})
##adding annotations
    total_counts_dict = dict(df[grouper].value_counts().sort_index())
    plot_dict={}
   
    for i in df[col].unique():
        val_counts = dict(df.loc[df[col] == i, grouper].value_counts())
        for k,v in val_counts.items():
            if k not in plot_dict:
                plot_dict[val_counts[k]] = (val_counts[k]/total_counts_dict[k])*100
            else:
                plot_dict[0]=0       
    for p in g.patches:
         height = p.get_height()
         if math.isnan(height):
            continue
         g.text(p.get_x()+p.get_width()/2.0,
         height + 0.5,
         f"{plot_dict[height]:.0f}%" , ha = "center", va = "bottom", fontsize = 8,weight="semibold",size="large")
 






In [ ]:
#We can also see which Type gets adopted faster based on given data'        


make_plot(train_ed,"Type",palette=sns.color_palette("Spectral_r"))



Based on the above plot, we can infer that Cat and Dog adoption increases sharply within a week of their listings.
But  a quarter of the total Cats and one-third of the Dogs still remain unadopted. There are other factors to consider like age, health etc as to why this is happening.

Now we can further analyse how Health factors like Vaccinated, Dewormed, Sterilized, Health affected Adoption Speeds.

In [ ]:
import matplotlib.pyplot as plt1 
plt1.figure(figsize=(25,15))
plt1.subplots_adjust(hspace=0.7,wspace=0.7)
plt1.subplot(2,2,1)
make_plot(df=train_ed,grouper="Vaccinated",title='Adoption Rate vs Vaccinated',palette='rainbow')
plt1.subplot(2,2,2)
make_plot(df=train_ed,grouper="Sterilized",title='Adoption Rate vs Sterilized',palette='Spectral')
plt1.subplot(2,2,3)
make_plot(df=train_ed,grouper="Dewormed",title='Adoption Rate vs Dewormed',palette='icefire')
plt1.subplot(2,2,4)
make_plot(df=train_ed,grouper="Health",title='Adoption Rate vs Health',palette='winter_r')
            
        
    
    

Based on the graphs, we can be sure that animals that are vaccinated in the earlier listing period are less likely to be adopted as compared to animals who aren't vaccinated. 
More than 30% of Dogs who are vaccinated are not even Adopted.
On the other hand, more than 75% of pets who aren't Vaccinated are Adopted.

People tend to prefer non.sterilized pets as we can see that almost 80% are adopted within 90 days of their listing.

It can be considered a general conception that people would prefer Healthy Cats and Dogs, and the data proves it too! 
The number of animals with injuries (Major or Minor) are significantly less and a major chunk of them also don't get adopted.


Now we analyse relationship for MaturitySize and Furlength with AdoptionSpeed.
First we look into MaturitySize for Cats and Dogs individually


In [ ]:
plt1.figure(figsize=(20,10))
plt1.subplots_adjust(hspace=0.7,wspace=0.7)
plt1.subplot(2,2,1)
df=train_ed.loc[train_ed['Type'] == "Cat"]
ax1=make_plot(df=df,grouper="MaturitySize",title='Adoption Rate vs MaturitySize for Cats',palette='summer')

plt1.subplot(2,2,2)
df=train_ed.loc[train_ed['Type'] == "Dog"]
ax2=make_plot(df=df,grouper="MaturitySize",title='Adoption Rate vs MaturitySize for Dogs',palette='winter')

plt1.subplot(2,2,3)

ax2=make_plot(df=train_ed,grouper="MaturitySize",title='Adoption Rate vs MaturitySize',palette='OrRd_r')

plt1.subplot(2,2,4)
ax2=make_plot(df=train_ed,col='Type',grouper='MaturitySize',palette='Set2',title='Counts per MaturitySize type')



There aren't any special observations except the fact that most common Maturity Size is Medium.
So it might be the case that there is bias in the data for number of observations. (Graph 4 shows that **medium** Maturity Size pets are higher in Number )


In [ ]:
plt1.figure(figsize=(20,10))
plt1.subplots_adjust(hspace=0.7,wspace=0.4)
plt1.subplot(2,2,1)
df=train_ed.loc[train_ed['Type'] == "Cat"]
ax1=make_plot(df=df,grouper="FurLength",title='Adoption Rate vs FurLength for Cats',palette='summer')

plt1.subplot(2,2,2)
df=train_ed.loc[train_ed['Type'] == "Dog"]
ax2=make_plot(df=df,grouper="FurLength",title='Adoption Rate vs FurLength for Dogs',palette='winter')

plt1.subplot(2,2,3)

ax2=make_plot(df=train_ed,grouper="FurLength",title='Adoption Rate vs FurLength',palette='OrRd_r')

plt1.subplot(2,2,4)
ax2 = make_plot(col='Type',grouper='FurLength',df=train_ed,palette='Set3',title='Counts per FurLength type')

Similar observations go for furlength feature **(Short Fur animals are most common in the listings, Hence introducing bias)**
So what we can do is, check for the same bias in the test set.

In [ ]:
plt1.figure(figsize=(20,7))
plt1.subplots_adjust(hspace=0.7,wspace=0.4)

plt1.subplot(1,2,1)
ax=make_plot(df=test,col='Type',grouper='MaturitySize',palette='OrRd_r',title='MaturitySize vs Count of pets per type for test set')
plt1.subplot(1,2,2)
ax=make_plot(df=test,col='Type',grouper='FurLength',palette='OrRd_r',title='FurLength vs Count of pets per type for test set')

Same observations are seen, i.e, Medium Sized or Small Furred pets are most common in the listings.
So to prevent bias, we'll have to do PCA or regularization to prevent overfitting in later stages.

We now see the effect of pet color on Adoption Speed

In [ ]:
color_labels.rename(columns={'Color3':'ColorID','ColorName3':'ColorName'},inplace = True)
palette_=['Black', 'Brown', '#FFFDD0', 'Gray', 'Gold', 'White', 'Yellow']
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
make_plot(df=train_ed,grouper='ColorName1',palette=palette_,col='Type',title='Counts for training set based on color')
# Now we check for test set
colors_dict = {k: v for k, v in zip(color_labels['ColorID'], color_labels['ColorName'])}
test["ColorName1_"] = test["Color1"].apply(lambda x: colors_dict[x] if x in colors_dict else '')
plt.subplot(1,2,2)
make_plot(df=test,grouper='ColorName1_',palette=palette_,col='Type',title='Counts for test set based on color')



The most common animals in both the datasets are Black and Brown in color.

We can now see how various colors affect adoption speed

In [ ]:
plt.figure(figsize=(18,8))
plt.subplots_adjust(hspace=0.9,wspace = 0.5)
# palette_=['Black', 'Brown', '#FFFDD0', 'Gray', 'Gold', 'White', 'Yellow']
for i,v in enumerate(train_ed["ColorName1"].unique()):
    plt.subplot(2,4,i+1)
    g=sns.countplot(x='AdoptionSpeed',data=train_ed[train_ed["ColorName1"]==v])
    g.set_title(v + " vs Adoption Speed")
    g.set_xticklabels(labels = train_ed["AdoptionSpeed"].unique(),rotation = -60,ha='left')
    colors_dict = dict(train_ed.loc[train_ed["ColorName1"] == v, "AdoptionSpeed"].value_counts())
    plot_dict={}
    for k,v in colors_dict.items():
        if k not in plot_dict:
                plot_dict[colors_dict[k]]=colors_dict[k]/sum(colors_dict.values())*100
        else:
            plot_dict[0]=0
    for x in g.patches:
        height = x.get_height()
        g.text(x.get_x() + x.get_width()/2,height,f"{plot_dict[height]:.0f}%",
               ha='center',va='bottom',fontsize = 8,weight="semibold",size="large")

Even though Black and Brown are the most common colors for pets in the dataset, There are more than 2000 black colored pets who are unadopted.

Let's also analyse AdoptionSpeed across various combinations of animal color. (ColorName1,ColorName2 and ColorName3)

In [ ]:
train_ed["All_Colors"] = train_ed["ColorName1"] + train_ed["ColorName2"] + train_ed["ColorName3"]
train_ed["All_Colors"].unique()

Let's see which colored animals are the highest in number

In [ ]:
#Selecting the top 5 values
top_5_colors = train_ed["All_Colors"].value_counts()[:5]
top_5_color = dict(top_5_colors)
top_5_color

These are the top 5 colors which are listed in the dataset.
Now let's analyse the effect of Fee on Adoption Speed

In [ ]:
sum_fee = train_ed["Fee"].value_counts()[:10]
sum_fee

In [ ]:
print(sum_fee.sum())

As we can see, more than 14,000 pets in the data are bought in under 300 Singapore Dollars. Most of the Animals (Around 12.5k) in the listing who are adopted are free of cost. Let's analyse the Adoption Speed Based on the Fee charged and Type for the Animal

In [ ]:
plt.figure(figsize=(18,8))
sns.scatterplot(x='AdoptionSpeed',y='Fee',data=train_ed,palette="summer",hue='Type')

From the plot, it is pretty clear that most pets within fee range below 1000 dollars  are adopted more. There is only 1 dog costing 3000 $ who also is adopted within 2-3 months. This basically means that generally people adopt pets who are mid-range expensive

We now analyse the relationship between Quantity and Fee

In [ ]:
plt.figure(figsize=(16, 10));
sns.scatterplot(x="Fee", y="Quantity", hue="Type",data=train_ed);
plt.title('Quantity of pets and Fee');

Based on the plot, Fees and Quantity have an inverted relationship. Lesser the number of pets, higher is the fee.

In [ ]:
train_ed.columns

We can check whether age of a pet(Cat or Dog) affects Adoption Speed or not

In [ ]:
plt.figure(figsize=(16, 10));
sns.scatterplot(x="AdoptionSpeed", y="Age", hue="Type",data=train_ed);
plt.title('Age vs Adoption speed for each Pet type');

Pets are generally adopted within 0-100 month range. A few older cats are adopted too. But there are dogs above 150 months who aren't adopted. As cats live longer than dogs, it would make sense to adopt an older cat than a dog.

Now we see how top 10 Breeds listed have an effect on Adoption Speed

In [ ]:
top_10_index = train_ed["MainBreed"].value_counts()[:10].index
top_10_vals = train_ed["MainBreed"].value_counts()[:10].values
top_10_breeds = { k:v for k,v in zip(top_10_index,top_10_vals)}
top_10_breeds

# plt.figure(figsize=(18,8))
# plt.subplots_adjust(hspace = 0.9,wspace = 0.7)
# count = 1
# for k,v in top_10_breeds.items():
#     plt.subplot(2,5,count)
#     g=sns.countplot(x='AdoptionSpeed',data=train_ed[train_ed["MainBreed"]==k])
#     g.set_title(k)
#     g.set_xticklabels(labels = train_ed["AdoptionSpeed"].unique(),rotation = -60, ha='left')
#     count = count + 1
    
